In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler 
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, r2_score, mean_squared_error
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
import matplotlib.pyplot as plt
from pathlib import Path
import os
import warnings
warnings.filterwarnings('ignore')

# Diretório onde o script está localizado
base_diretório = Path(os.getcwd())
base_diretório = base_diretório.parent
caminho = base_diretório / 'Bases' / 'Ethereum Historical Data.csv'

# Ler arquivo da base
acao = pd.read_csv(caminho)

# Formatar data
acao['Date'] = pd.to_datetime(acao['Date'], format='%m/%d/%Y')

# Filtrar a data em que vamos puxar os dados
# acao = acao[acao['Date'].dt.year >= 2023]

# Converter as colunas referentes a dinheiro para float
colunas_dinheiro = ['Price', 'Open', 'High', 'Low']
for coluna in colunas_dinheiro:
    if acao[coluna].dtype == 'object':  
        acao[coluna] = pd.to_numeric(acao[coluna].str.replace(',', ''))

# Converter a coluna 'Change %' para float
if acao['Change %'].dtype == 'object':  
    acao['Change %'] = pd.to_numeric(acao['Change %'].str.replace('%', ''))

def converter_vol_para_numero(valor):
    if isinstance(valor, str):  # Verifica se o valor é uma string
        if 'K' in valor:
            return float(valor.replace('K', '').replace(',', '')) * 1000
        elif 'M' in valor:
            return float(valor.replace('M', '').replace(',', '')) * 1000000
        elif 'B' in valor:
            return float(valor.replace('B', '').replace(',', '')) * 1000000000
        else:
            return float(valor.replace(',', ''))
    else:
        return valor  # Retorna o valor diretamente se já for numérico

# Aplicar a função na coluna 'Vol.'
acao['Vol.'] = acao['Vol.'].apply(converter_vol_para_numero)

# Ordenar data das bases
acao = acao.sort_values(by='Date', ascending=True)

# Renomear coluna
acao = acao.rename(columns={'Vol.': 'Volume'})


# Transforma o array de uma dimensão (array([price1, price2, price3, ...])) 
# em um array 2D com uma coluna (array([[price1], [price2], [price3], ...]))
cotacao = acao['Price'].to_numpy().reshape(-1, 1)

# Separar 80% da base para treinamento
tamanho_dados_treinamento = int(len(cotacao) * 0.8)

# Escalar os dados entre 0 e 1
escalador = MinMaxScaler(feature_range=(0, 1))
dados_entre_0_e_1_treinamento = escalador.fit_transform(cotacao[0:tamanho_dados_treinamento, :])
dados_entre_0_e_1_teste = escalador.transform(cotacao[tamanho_dados_treinamento:, :])

dados_entre_0_e_1 = np.concatenate((dados_entre_0_e_1_treinamento, dados_entre_0_e_1_teste), axis=0)

dados_para_treinamento = dados_entre_0_e_1[0:tamanho_dados_treinamento, :]
treinamento_x, treinamento_y = [], []

for i in range(60, len(dados_para_treinamento)):
    treinamento_x.append(dados_para_treinamento[i - 60:i, 0])
    treinamento_y.append(dados_para_treinamento[i, 0])

# Transformando listas em arrays
treinamento_x, treinamento_y = np.array(treinamento_x), np.array(treinamento_y)
treinamento_x = treinamento_x.reshape(treinamento_x.shape[0], treinamento_x.shape[1], 1)


# produto cartesiano
# Parâmetros para otimização
neurons_options = [50, 60, 70, 80, 90, 100]
batch_size_options = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120]
epochs_options = [10, 20]

results = []

# 32x
idx = 1
# while idx != 33:
for neurons in neurons_options:
    for batch_size in batch_size_options:
        for epochs in epochs_options:
                
            print(f"Combinação {idx}: neuronios={neurons}, batch_size={batch_size}, epochs_options={epochs}")
                
            # Construindo o modelo
            modelo = Sequential()
            modelo.add(LSTM(neurons, return_sequences=True, input_shape=(treinamento_x.shape[1], 1)))
            modelo.add(LSTM(neurons // 2, return_sequences=False))
            modelo.add(Dense((neurons // 2)//2))
            modelo.add(Dense(1))
            # Compilando o modelo
            modelo.compile(optimizer="adam", loss="mean_squared_error")
                
            # Treinando o modelo
            modelo.fit(treinamento_x, treinamento_y, batch_size=batch_size, epochs=epochs, verbose=0)
            # Criando dados de teste
            dados_teste = dados_entre_0_e_1[tamanho_dados_treinamento - 60:, :]
            teste_x = []
            teste_y = cotacao[tamanho_dados_treinamento:, :]
            for i in range(60, len(dados_teste)):
                teste_x.append(dados_teste[i - 60:i, 0])
            teste_x = np.array(teste_x).reshape(len(teste_x), 60, 1)
            # Pegando predições do modelo
            predicoes = modelo.predict(teste_x)
            predicoes = escalador.inverse_transform(predicoes)
                
            # Criar df
            df_previsao = pd.DataFrame({"Date": acao['Date'].iloc[tamanho_dados_treinamento:],
                                            "Price": acao['Price'].iloc[tamanho_dados_treinamento:],
                                            "predicoes": predicoes.reshape(len(predicoes))
                                        })
                
            df_previsao.set_index('Date', inplace=True)
                
            df_previsao['Semana'] = ((df_previsao.index - df_previsao.index.min()).days // 7) + 1
                
            #========================== SEMANAL ==========================
            df_semana = df_previsao
            # Agrupar por semana e calcular o preço médio semanal para real e previsões
            df_semana = df_previsao.groupby('Semana').agg({
                'Price': 'mean',        # Preço real médio por semana
                'predicoes': 'mean'     # Previsão média por semana
            })
            # Calcular a variação percentual semanal para Price e Predicoes
            df_semana['Variação Real (%)'] = df_semana['Price'].pct_change() * 100
            df_semana['Variação Prevista (%)'] = df_semana['predicoes'].pct_change() * 100
            # Remover NaN (primeira linha não terá variação por não ter valor anterior)
            df_semana = df_semana.dropna()
            # Calcular se a direção do valor previsto foi igual ao real (1 = acertou | 0 = errou)
            df_semana['direcao_correta'] = np.where(
                (df_semana['Variação Real (%)'] > 0) & (df_semana['Variação Prevista (%)'] > 0) |
                (df_semana['Variação Real (%)'] < 0) & (df_semana['Variação Prevista (%)'] < 0),
                1, 0
            )
            # Calcular a métrica de Erro de Direção (Directional Accuracy)
            directional_accuracy_semanal = df_semana['direcao_correta'].mean()
            directional_accuracy_semanal = round(directional_accuracy_semanal * 100, 2)
            # Avaliando o modelo
            mae = mean_absolute_error(teste_y, predicoes)
            mape = mean_absolute_percentage_error(teste_y, predicoes)
            rmse = mean_squared_error(teste_y, predicoes, squared=False)
            # Armazenando os resultados
            results.append({
                'neurons': neurons,
                'batch_size': batch_size,
                'epochs': epochs,
                'MAE': mae,
                'MAPE': mape,
                'RMSE': rmse,
                'directional_accuracy_semanal': directional_accuracy_semanal
            })
                
            idx+=1

# Convertendo resultados em DataFrame
results_df = pd.DataFrame(results)

# Exibindo resultados
print(results_df)

# Gráfico do melhor modelo (opcional)
best_model = results_df.loc[results_df['directional_accuracy_semanal'].idxmax()]

print(f"\nMelhor Modelo: Neurônios={best_model['neurons']}, batch_size={best_model['batch_size']}, Epochs={best_model['epochs']}")


Combinação 1: neuronios=50, batch_size=10, epochs_options=10
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step
Combinação 2: neuronios=50, batch_size=10, epochs_options=20
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step
Combinação 3: neuronios=50, batch_size=20, epochs_options=10
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step
Combinação 4: neuronios=50, batch_size=20, epochs_options=20
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step
Combinação 5: neuronios=50, batch_size=30, epochs_options=10
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step
Combinação 6: neuronios=50, batch_size=30, epochs_options=20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Combinação 7: neuronios=50, batch_size=40, epochs_options=10
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step
Combinação 8: neuronios=50, batch_size=40, epochs_options=20
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step
Combinação 9: neuronios=50, batch_size=50, epochs_options=10
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step
Combinação 10: neuronios=50, batch_size=50, epochs_options=20
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s

In [3]:
results_df.to_excel("Resultados testes parametros/Ethereum - Resultado geral.xlsx", engine="openpyxl", index=False)